In [ ]:

get_ipython().system('pip install snowflake-connector-python')
get_ipython().system('pip install tldextract')
get_ipython().system('pip install py2neo')

# In[175]:

import numpy as np
import os
import requests
import snowflake.connector
from py2neo import Graph
from py2neo.bulk import merge_nodes, merge_relationships
from urllib.parse import urlparse

#[Environment Configs]
client_id = "d0afa05e7ab642acb0497f018b601b86"
client_secret = "ikdU3eOtXRmMR9oWlEnP0W0jAVZYNMBF2FUwsuwtHkZ37Oh4IZZkso3tFdsbfdmO"
os.environ["HTTP_PROXY"] = "http://d161560-001.dc.gs.com:8899"
os.environ["HTTPS_PROXY"] = "http://d161560-001.dc.gs.com:8899"

#[Credentials]
host = "xplore.graph-x.site.gs.com"
port = "7687"
username = "neo4j"
password = "xplorer"
graph = Graph(host=host, port=port, auth=(username, password))
BATCHES = 100


#### Libraries & Shared Code

In [ ]:
def get_oauth_tokens(client_id: str, client_secret: str):
    _IDFS_URL = "https://idfs.gs.com/as/token.oauth2"
    response = requests.post(
        url=_IDFS_URL,
        params={"access_token_manager_id": "JwtSnowflake"},
        # This must be set to JwtSnowflake to generate Snowflake-compatible OAuth Token
        data={"grant_type": "client_credentials", "scope": "SESSION:ROLE-ANY"},
        auth=(client_id, client_secret),
    )
    response = response.json()
    if "access_token" not in response.keys():
        raise PermissionError(f"Not authorized to access OAuth Tokens: {response}")
    return response["access_token"]


def connect_snowflake(user: str, database: str, token: str) -> snowflake.connector.cursor:
    """Connect to Snowflake and return an authorized cursor. Below uses Catalyst connections

    :param user: Client ID from OAuth2 Authentication
    :param database: Name of database to connect to
    :param token: Generated OAuth Token. Please either generate and save to a tmp file in a separate thread OR use subprocess module to call the underlying subsystem and capture the output
    :return: Snowflake Cursor
    """
    snow_connector = snowflake.connector.connect(
        user=user,
        authenticator="oauth",
        account="sfamdprvtawseast1d01.goldman.us-east-1.aws.privatelink",
        role="AMD_CATALYST_DATA_RW",
        warehouse="AMD_CATALYST_RW",
        database=database,
        ocsp_fail_open=False,
        token=token,
    )
    cursor = snow_connector.cursor()
    return cursor


def disconnect(cursor: snowflake.connector.cursor) -> None:
    try:
        cursor.close()
    except:
        raise Exception("Failed to close cursor")


oauth_token = get_oauth_tokens(client_id, client_secret)
cursor = connect_snowflake(client_id, "AMD_CATALYST_DB", oauth_token)

### Sourcescrub Person

df_from_database = cursor.execute('SELECT columns FROM database).fetch_pandas_all()

source: database



In [ ]:
sourcescrub_persons_df = cursor.execute(
    'SELECT ID, FIRSTNAME, LASTNAME, LINKEDIN, TWITTER, FACEBOOK, WEBSITE, PERSONALEMAIL, CITY, STATE, COUNTRY FROM L2.GRAPH_SS_PERSON').fetch_pandas_all()

# In[249]:


sourcescrub_persons_df['NAME'] = sourcescrub_persons_df['FIRSTNAME'] + ' ' + sourcescrub_persons_df['LASTNAME']
sourcescrub_persons_df.drop(columns=['FIRSTNAME', 'LASTNAME'], inplace=True)
sourcescrub_persons_df['ID'] = 'sourcescrub-' + sourcescrub_persons_df['ID'].astype(str)
sourcescrub_persons_df.rename(columns={'ID': 'SOURCE_ID', "PERSONALEMAIL": "EMAIL"}, inplace=True)

# In[ ]:


sourcescrub_persons_df['LINKEDIN'] = sourcescrub_persons_df['LINKEDIN'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
sourcescrub_persons_df['TWITTER'] = sourcescrub_persons_df['TWITTER'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
sourcescrub_persons_df['FACEBOOK'] = sourcescrub_persons_df['FACEBOOK'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
sourcescrub_persons_df['WEBSITE'] = sourcescrub_persons_df['WEBSITE'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
sourcescrub_persons_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# In[ ]:


sourcescrub_persons_df.head()

# In[ ]:


sourcescrub_persons_df_website = np.array_split(
    sourcescrub_persons_df[sourcescrub_persons_df['WEBSITE'].isna() == False][['WEBSITE']].rename(
        columns={'WEBSITE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_persons_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

sourcescrub_person_df_linkedin = np.array_split(
    sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(
        columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_person_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

sourcescrub_person_df_twitter = np.array_split(
    sourcescrub_persons_df[sourcescrub_persons_df['TWITTER'].isna() == False][['TWITTER']].rename(
        columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_person_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

sourcescrub_person_df_facebook = np.array_split(
    sourcescrub_persons_df[sourcescrub_persons_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(
        columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_person_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

sourcescrub_person_df_email = np.array_split(
    sourcescrub_persons_df[sourcescrub_persons_df['EMAIL'].isna() == False][['EMAIL']].rename(
        columns={'EMAIL': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_person_df_email:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "EmailURL"), "URL"),
    )

# In[ ]:


sourcescrub_persons = np.array_split(
    [{k: v for k, v in x.items() if v == v} for x in sourcescrub_persons_df.to_dict('records')], BATCHES)
for chunk_df in sourcescrub_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "SourcescrubPerson"), "SOURCE_ID"),
    )

# In[ ]:


for identifier in ['WEBSITE', 'LINKEDIN', 'TWITTER', 'FACEBOOK']:
    sourcescrub_identifier_relationships = sourcescrub_persons_df.loc[sourcescrub_persons_df[identifier] != np.nan][
        ['SOURCE_ID', identifier]]
    sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [0.5] * len(sourcescrub_identifier_relationships))

    df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Person", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

# ## PERSONTOCOMPANY

# In[ ]:


sourcescrub_p2c_df = cursor.execute(
    'SELECT COMPANYID, PERSONID, TITLE, ROLE, STARTDATE, ENDDATE FROM L1.SOURCESCRUB_PERSONTOCOMPANY').fetch_pandas_all()

# In[ ]:


sourcescrub_p2c_df['PERSONID'] = 'sourcescrub-' + sourcescrub_p2c_df['PERSONID'].astype(str)
sourcescrub_p2c_df['COMPANYID'] = 'sourcescrub-' + sourcescrub_p2c_df['COMPANYID'].astype(str)
sourcescrub_p2c_df.rename(
    columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'},
    inplace=True)
sourcescrub_p2c_df.START_DATE = sourcescrub_p2c_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_p2c_df.END_DATE = sourcescrub_p2c_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_p2c_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# In[ ]:


sourcescrub_p2c_df.head()

# In[ ]:


# sourcescrub_p2c_df.drop(columns = "WEIGHT", inplace = True)


# In[ ]:


sourcescrub_p2c_df.insert(2, 'WEIGHT', [0.5] * len(sourcescrub_p2c_df))
sourcescrub_p2c_df['P2C_DETAILS'] = [{k: v for k, v in x.items() if v == v} for x in
                                     sourcescrub_p2c_df.to_dict('records')]

sourcescrub_curr_emp_df = sourcescrub_p2c_df[sourcescrub_p2c_df['END_DATE'].isna() == True].loc[:,
                          ["COMPANY_ID", "P2C_DETAILS", "PERSON_ID"]]
sourcescrub_fmr_emp_df = sourcescrub_p2c_df[sourcescrub_p2c_df['END_DATE'].isna() == False].loc[:,
                         ["COMPANY_ID", "P2C_DETAILS", "PERSON_ID"]]

# In[ ]:


sourcescrub_p2c_df.head()

# In[ ]:


sourcescrub_curr_emp_df.head()

# In[ ]:


# current employment
sourcescrub_curr_emp = np.array_split(sourcescrub_curr_emp_df, BATCHES)
for chunk_df in sourcescrub_curr_emp:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_EMPLOYMENT",
        start_node_key=("Company", "SOURCE_ID"),
        end_node_key=("Person", "SOURCE_ID")
    )
# previous employment    
sourcescrub_fmr_emp = np.array_split(sourcescrub_fmr_emp_df, BATCHES)
for chunk_df in sourcescrub_fmr_emp:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="FORMER_EMPLOYMENT",
        start_node_key=("Company", "SOURCE_ID"),
        end_node_key=("Person", "SOURCE_ID")
    )

# ### SourceScrub Boardmembers

# In[ ]:


ss_bm_df = cursor.execute(
    'SELECT PERSONID, COMPANYID, EMAIL, FIRSTNAME, LASTNAME, TITLE, LINKEDIN, STARTDATE, ENDDATE FROM AMD_CATALYST_DB.L2_NEW.SOURCESCRUB_BOARDMEMBERS WHERE LINKEDIN IS NOT NULL').fetch_pandas_all()
ss_bm_df.head()

# In[ ]:


ss_bm_df['NAME'] = ss_bm_df['FIRSTNAME'] + ' ' + ss_bm_df['LASTNAME']
ss_bm_df.drop(columns=['FIRSTNAME', 'LASTNAME'], inplace=True)
ss_bm_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
ss_bm_df.rename(
    columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'},
    inplace=True)
ss_bm_df.START_DATE = ss_bm_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
ss_bm_df.END_DATE = ss_bm_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
ss_bm_df.insert(2, 'WEIGHT', [0.5] * len(ss_bm_df))
ss_bm_df['BOARDMEMBERS_INFO'] = [{k: v for k, v in x.items() if v == v} for x in ss_bm_df.to_dict('records')]

# In[ ]:


ss_bm_df.head()

# In[ ]:


ss_curr_bm_df = ss_bm_df[ss_bm_df['END_DATE'].isna() == True].loc[:, ["COMPANY_ID", "BOARDMEMBERS_INFO", "PERSON_ID"]]
ss_fmr_bm_df = ss_bm_df[ss_bm_df['END_DATE'].isna() == False].loc[:, ["COMPANY_ID", "BOARDMEMBERS_INFO", "PERSON_ID"]]

# In[ ]:


ss_fmr_bm_df.head()

# In[ ]:


ss_curr_bm = np.array_split(ss_curr_bm_df, BATCHES)
for chunk_df in ss_curr_bm:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_BOARDMEMBER"
start_node_key = ("Company", "SOURCE_ID")
end_node_key = ("Person", "SOURCE_ID"))


# In[ ]:

ss_fmr_bm = np.array_split(ss_fmr_bm_df, BATCHES)
for chunk_df in ss_fmr_bm:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="FORMER_BOARDMEMBER"
start_node_key = ("Company", "SOURCE_ID")
end_node_key = ("Person", "SOURCE_ID"))


# ### SourceCrub Executives

# In[ ]:

ss_exec_df = cursor.execute(
    'SELECT * FROM AMD_CATALYST_DB.L2_NEW.SOURCESCRUB_EXECUTIVES WHERE LINKEDIN IS NOT NULL').fetch_pandas_all()
# ss_exec_df.head()

# In[ ]:

ss_exec_df['NAME'] = ss_exec_df['FIRSTNAME'] + ' ' + ss_exec_df['LASTNAME']
ss_exec_df.drop(columns=['FIRSTNAME', 'LASTNAME'], inplace=True)

ss_exec_df['LINKEDIN'] = ss_exec_df['LINKEDIN'].apply(
    lambda x: (str(urlparse(x).netloc or '').lstrip('www.') + str(urlparse(x).path or '')))

ss_exec_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

ss_exec_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE',
                           'ENDDATE': 'END_DATE'}, inplace=True)

ss_exec_df.START_DATE = ss_exec_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
ss_exec_df.END_DATE = ss_exec_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)

# In[ ]:

ss_exec_df.head(3)

# In[ ]:

ss_exec_df.insert(2, 'WEIGHT', [0.5] * len(ss_exec_df))
ss_exec_df['EXECUTIVE_INFO'] = [{k: v for k, v in x.items() if v == v} for x in ss_exec_df.to_dict('records')]

# In[ ]:

ss_curr_exec_df = ss_exec_df[ss_exec_df['END_DATE'].isna() == True].loc[:,
                  ["COMPANY_ID", "EXECUTIVE_INFO", "PERSON_ID"]]
ss_fmr_exec_df = ss_exec_df[ss_exec_df['END_DATE'].isna() == False].loc[:,
                 ["COMPANY_ID", "EXECUTIVE_INFO", "PERSON_ID"]]

# In[ ]:

ss_curr_exec_df.head()

# In[ ]:

ss_curr_exec = np.array_split(ss_curr_exec_df, BATCHES)
for chunk_df in ss_curr_exec:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_EXECUTIVE"
start_node_key = ("Company", "SOURCE_ID")
end_node_key = ("Person", "SOURCE_ID"))


# In[ ]:

ss_fmr_exec = np.array_split(ss_fmr_exec_df, BATCHES)
for chunk_df in ss_fmr_exec:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="FORMER_EXECUTIVE"
start_node_key = ("Company", "SOURCE_ID")
end_node_key = ("Person", "SOURCE_ID"))

In [ ]:

# ### Crunchbase People

# In[11]:

cb_persons_df = cursor.execute(
    'SELECT "CB URL", CITY, "COUNTRY CODE", "STATE CODE", "FACEBOOK URL", GENDER, "LINKEDIN URL", NAME, "TWITTER URL", UUID, "WEBSITE URL" FROM L2.CRUNCHBASE_PEOPLE_TWO').fetch_pandas_all()
cb_persons_df.head()

# In[12]:

cb_persons_df.rename(columns={
    'UUID': 'PERSON_ID',
    'TWITTER URL': 'TWITTER',
    'FACEBOOK URL': 'FACEBOOK',
    'LINKEDIN URL': 'LINKEDIN',
    'CB URL': 'CRUNCHBASE',
    'WEBSITE URL': 'WEBSITE',
    'STATE CODE': 'STATE',
    'COUNTRY CODE': 'COUNTRY', }, inplace=True)

cb_persons_df['LOCATION'] = cb_persons_df['CITY'] + ', ' + cb_persons_df['COUNTRY'] + ', ' + \
                            cb_persons_df['STATE']
cb_persons_df.drop(columns=['CITY', 'COUNTRY', 'STATE'], inplace=True)

cb_persons_df.replace({None: ''}, inplace=True)

cb_persons_df['PERSON_ID'] = 'crunchbase-' + cb_persons_df['PERSON_ID'].astype(str)

cb_persons_df['LINKEDIN'] = cb_persons_df['LINKEDIN'].apply(lambda x: (
        str(urlparse(x.replace('[', '').replace(']', '')).netloc or '') + str(
    urlparse(x.replace('[', '').replace(']', '')).path or '')).lstrip('www.'))
cb_persons_df['TWITTER'] = cb_persons_df['TWITTER'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
cb_persons_df['FACEBOOK'] = cb_persons_df['FACEBOOK'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
cb_persons_df['CRUNCHBASE'] = cb_persons_df['CRUNCHBASE'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
cb_persons_df['WEBSITE'] = cb_persons_df['WEBSITE'].apply(
    lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))

cb_persons_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# In[13]:

display(cb_persons_df.head())

# In[17]:

cb_persons_df.rename(columns={'PERSON_ID': 'SOURCE_ID'}, inplace=True)

# In[14]:

cb_persons_df_website = np.array_split(
    cb_persons_df[cb_persons_df['WEBSITE'].isna() == False][['WEBSITE']].rename(
        columns={'WEBSITE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_persons_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

# In[22]:

cb_persons_df_linkedin = np.array_split(
    cb_persons_df[cb_persons_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(
        columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_persons_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

# In[ ]:

cb_persons_df_twitter = np.array_split(
    cb_persons_df[cb_persons_df['TWITTER'].isna() == False][['TWITTER']].rename(
        columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_persons_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

# In[ ]:

cb_persons_df_facebook = np.array_split(
    cb_persons_df[cb_persons_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(
        columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_persons_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

# In[ ]:

cb_persons_df_crunchbase = np.array_split(
    cb_persons_df[cb_persons_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(
        columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_persons_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

# In[20]:

# creating people node
cb_persons = np.array_split(
    [{k: v for k, v in x.items() if v == v} for x in cb_persons_df.to_dict('records')], BATCHES)
for chunk_df in cb_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "CrunchbasePerson"), "SOURCE_ID"),
    )

# In[ ]:

for identifier in ['WEBSITE', 'LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    crunchbase_identifier_relationships = cb_persons.loc[cb_persons[identifier] != np.nan][
        ['SOURCE_ID', identifier]]
    crunchbase_identifier_relationships.insert(1, 'WEIGHT',
                                               [[0.5]] * len(crunchbase_identifier_relationships))

    df_list = np.array_split(crunchbase_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Person", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

# ### CrunchBase Person to Company

# In[126]:

cb_p2c_df = cursor.execute(
    'SELECT "JOB TYPE", "TITLE", "PERSON UUID", "PERSON NAME", "ORG NAME", "ORG UUID", "STARTED ON", "ENDED ON" FROM L2.GRAPH_CRUNCHBASE_CURRENT_JOBS').fetch_pandas_all()
# cb_p2c_df.head()

# In[128]:

np.sum(cb_p2c_df['ENDED ON'] == None)

# In[106]:

cb_p2c_df.rename(columns=
                 {'JOB TYPE': 'JOB_TYPE',
                  'PERSON UUID': 'PERSON_ID',
                  'PERSON NAME': 'PERSON_NAME',
                  'ORG NAME': 'COMPANY_NAME',
                  'ORG UUID': 'COMPANY_ID',
                  'STARTED ON': 'START_DATE',
                  'ENDED ON': 'END_DATE'}, inplace=True)

#cb_p2c_df.replace({None: ''}, inplace=True)

cb_p2c_df['COMPANY_ID'] = 'crunchbase-' + cb_p2c_df['COMPANY_ID'].astype(str)
cb_p2c_df['PERSON_ID'] = 'crunchbase-' + cb_p2c_df['PERSON_ID'].astype(str)
cb_p2c_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

cb_p2c_df.START_DATE = cb_p2c_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
cb_p2c_df.END_DATE = cb_p2c_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
cb_p2c_df.insert(2, 'WEIGHT', [0.5] * len(cb_p2c_df))

# In[107]:

cb_p2c_df.head()

# In[108]:

# cb_p2c_df.drop(columns = "WEIGHT", inplace = True)
# add statement like if Weight not exist add weight

# In[109]:

cb_p2c_curr_df = cb_p2c_df[cb_p2c_df['END_DATE'].isna()]
cb_p2c_fmr_df = cb_p2c_df[cb_p2c_df['END_DATE'].isna() == False]

# In[110]:

cb_p2c_curr_df['CURRENT_EMP_DETAILS'] = [{k: v for k, v in x.items() if v == v} for x in
                                         cb_p2c_curr_df.to_dict('records')]
cb_curr_emp_df = cb_p2c_curr_df[["COMPANY_ID", "CURRENT_EMP_DETAILS", "PERSON_ID"]]

# In[112]:

cb_curr_emp_df.head()

# In[113]:

cb_fmr_emp_df.head()

# In[118]:

# cb_fmr_emp_df.head()

# In[116]:

cb_curr_emp = np.array_split(cb_curr_emp_df, BATCHES)
for chunk_df in cb_curr_emp:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_EMPLOYMENT",
        start_node_key=("Company", "SOURCE_ID"),
        end_node_key=("Person", "SOURCE_ID"))


cb_event_appear_df = cursor.execute(
    'SELECT * FROM L2_NEW.GRAPH_CRUNCHBASE_EVENT_PARTICIPANTS_PERSONS').fetch_pandas_all()
cb_event_appear_df.head()

# In[135]:

print("Is Unique: ", cb_event_appear_df.EVENT_UUID.is_unique)

# In[164]:

cb_event_appear_df.rename(columns={
    'PARTICIPANT_UUID': 'PERSON_ID',
    "APPEARANCE_TYPE": "TYPE",
    "EVENT_UUID": "EVENT_ID",
    'PARTICIPANT NAME': 'PERSON_NAME'}, inplace=True)

cb_event_appear_df['PERSON_ID'] = 'crunchbase-' + cb_event_appear_df['PERSON_ID'].astype(str)
cb_event_appear_df['EVENT_ID'] = 'crunchbase-' + cb_event_appear_df['EVENT_ID'].astype(str)

cb_event_appear_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# In[165]:

cb_event_appear_df.head()

# In[166]:

if 'WEIGHT' not in list(cb_event_appear_df.columns):
    cb_event_appear_df.insert(0, 'WEIGHT', [0.5] * len(cb_event_appear_df))

# In[167]:

cb_event_appear_df['PARTICIPANT_INFO'] = [{k: v for k, v in x.items() if v == v} for x in
                                          cb_event_appear_df.to_dict('records')]

# In[169]:

cb_event_appear_df['PARTICIPANT_INFO'] = [{k: v for k, v in x.items() if v == v} for x in
                                          cb_event_appear_df.to_dict('records')]
cb_event_appear_df_final = cb_event_appear_df[["PERSON_ID", "PARTICIPANT_INFO", "EVENT_ID"]]

# In[171]:

cb_event_appear_df_final.head()

# In[213]:

cb_event_appear = np.array_split(cb_event_appear_df_final, BATCHES)

for chunk_df in cb_event_appear:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="ATTEND_EVENT",
        start_node_key=("Person", "SOURCE_ID"),
        end_node_key=("Event", "SOURCE_ID"))

# ### CrunchBase Degree
# ##### Creating the school node

# In[245]:

univ_df = cursor.execute(
    'SELECT "INSTITUTION NAME", "INSTITUTION UUID" FROM L2.GRAPH_CRUNCHBASE_DEGREES').fetch_pandas_all()
univ_df.rename(columns={
    'INSTITUTION NAME': 'SCHOOL_NAME',
    'INSTITUTION UUID': 'SOURCE_ID'}, inplace=True)

univs = np.array_split([{k: v for k, v in x.items() if v == v} for x in univ_df.to_dict('records')],
                       BATCHES)

# In[247]:

for chunk_df in univs:
    merge_nodes(
        graph.auto(),
        chunk_df,
        ("School", "SOURCE_ID"),
    )

# ##### Creating relationship

# In[256]:

cb_degrees_df = cursor.execute(
    'SELECT "CB URL", "DEGREE TYPE", "PERSON NAME", "PERSON UUID", "SUBJECT", "INSTITUTION NAME", "INSTITUTION UUID", "STARTED ON", "COMPLETED ON" FROM L2.GRAPH_CRUNCHBASE_DEGREES').fetch_pandas_all()
cb_degrees_df.head()

# In[257]:

cb_degrees_df.rename(columns={
    'PERSON UUID': 'PERSON_ID',
    "CB URL": "CRUNCHBASE",
    'PERSON NAME': 'PERSON_NAME',
    'DEGREE TYPE': 'DEGREE',
    'INSTITUTION NAME': 'SCHOOL_NAME',
    'INSTITUTION UUID': 'SCHOOL_ID',
    'STARTED ON': 'START_DATE',
    'COMPLETED ON': 'END_DATE'}, inplace=True)

cb_degrees_df['PERSON_ID'] = 'crunchbase-' + cb_degrees_df['PERSON_ID'].astype(str)
cb_degrees_df['SCHOOL_ID'] = 'crunchbase-' + cb_degrees_df['SCHOOL_ID'].astype(str)

cb_degrees_df.START_DATE = cb_degrees_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
cb_degrees_df.END_DATE = cb_degrees_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
cb_degrees_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# In[258]:

if 'WEIGHT' not in list(cb_degrees_df.columns):
    cb_degrees_df.head().insert(0, 'WEIGHT', [0.5] * len(cb_degrees_df.head()))
cb_degrees_df['EDUCATION_HISTORY'] = [{k: v for k, v in x.items() if v == v} for x in
                                      cb_degrees_df.to_dict('records')]
cb_degrees_df.head()

# In[264]:

cb_degrees_df.rename(columns={
    'PERSON_ID': 'SOURCE_ID'}, inplace=True)
cb_degrees_df.head()

# In[ ]:

for identifier in ['CRUNCHBASE']:
    cb_identifier_relationships = cb_degrees_df.loc[cb_degrees_df[identifier] != np.nan][
        ['SOURCE_ID', identifier]]
    cb_identifier_relationships.insert(1, 'WEIGHT', [0.5] * len(cb_identifier_relationships))

    df_list = np.array_split(cb_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Person", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

# In[ ]:

cb_degrees_df_final = cb_degrees_df[['PERSON_ID', 'EDUCATION_HISTORY', 'SCHOOL_ID']]

# In[ ]:

cb_degrees_df_final.head()

# In[ ]:

cb_degree = np.array_split(cb_degrees_df_final, BATCHES)

for chunk_df in cb_degrees:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="ATTEND_SCHOOL",
        start_node_key=("Person", "SOURCE_ID"),
        end_node_key=("School", "SOURCE_ID"))

# In[ ]:

